In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tensorflow as tf

from skimage import color, exposure, filters, io, morphology, util
from math import sqrt
from numpy import loadtxt
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Reshape
from keras.optimizers import SGD

### A. _Fully Connected Neural Networks_

1. Lakukan klasifikasi untuk dataset cifar menggunakan **_Fully Connected Neural Networks_** dengan arsitektur dasar sebagai berikut:
    - _Fully Connected Layer_ ($1000$ _hidden node_).
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Fully Connected Layer_ ($1000$ _hidden node_)
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Output Layer_ (10 kelas).
    - _Activation Layer_ ($\text{Softmax function}$).
    - _Classification Result_.

Fitur yang digunakan bebas (_handcrafted_/_non-convolution_). Hitung metrik ($\text{accuracy}$, $\text{sensitivity}$, $\text{specificity}$, dan $f_{1}\text{ score}$) untuk data train dan data test.

#### Mendefinisikan Fungsi Bantuan
Untuk memudahkan implementasi, berikut beberapa fungsi bantuan yang akan dipakai

##### Menghitung $\text{recall}$ atau $\text{sensitivity}$
$$\text{recall} = \frac{t_p}{t_p+t_n}$$

In [3]:
def recall(y_true, y_pred): 
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  return true_positives / (possible_positives + K.epsilon())

##### Menghitung $\text{precision}$
$$\text{precision} = \frac{t_p}{t_p + f_p}$$

In [4]:
def precision(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  return true_positives / (predicted_positives + K.epsilon())

##### Menghitung $\text{specificity}$
$$\text{specificity} = \frac{t_n}{t_n + f_p}$$

In [5]:
def specificity(y_true, y_pred):
  true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
  possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
  return true_negatives / (possible_negatives + K.epsilon())

##### Menghitung $f_{1}\text{ Score}$
Kita tau bahwa
$$f_{\beta} = (1 + \beta^{2}) \cdot \frac{\text{precition} \cdot \text{recall}}{(\beta^{2} \cdot \text{precision}) + \text{recall}}$$
Sehingga 
$$f_{1} = (1 + 1^{2}) \cdot \frac{\text{precition} \cdot \text{recall}}{(1^{2} \cdot \text{precision}) + \text{recall}}$$
$$f_{1} = (2) \cdot \frac{\text{precition} \cdot \text{recall}}{(\text{precision}) + \text{recall}}$$



In [6]:
def f1_score(y_true, y_pred):
  precision_val = precision(y_true, y_pred)
  recall_val = recall(y_true, y_pred)
  return 2*((precision_val * recall_val)/(precision_val + recall_val + K.epsilon()))

**n.b.: Semua fungsi pada bagian penyebut ditambah epsilon untuk menghindari dibagi $0$ dikarenakan memasukkan input sebanyak $0$**

##### Fungsi Bantuan untuk Melakukan Prediksi dan Menampilkan Metric

In [7]:
def predict_and_get_metrics(model, x_test, y_test, label):
  _, accuracy, sensitivity, specificity, score_f1 = model.evaluate(x_test, y_test, verbose=0)
  return pd.DataFrame([[accuracy, sensitivity, specificity, score_f1]], columns=['Accuracy', 'Sensitivity', 'Specificity', 'F1 Score'], index=[label])

def compare_train_and_test(model, x_train, y_train, x_test, y_test, extra_label=''):
  metric_train = predict_and_get_metrics(model, x_train, y_train, 'Data Train %s' % (extra_label))
  metric_test = predict_and_get_metrics(model, x_test, y_test, 'Data Test %s' % (extra_label))
  return pd.concat([metric_train, metric_test])

##### Fungsi Bantuan untuk _Load Dataset_

In [8]:
def load_dataset():
	(trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()
	trainY = tf.keras.utils.to_categorical(trainY) ## Di ubah ke matrix persegi representasi binari
	testY = tf.keras.utils.to_categorical(testY) ## Di ubah ke matrix persegi representasi binari
	return trainX, trainY, testX, testY

#### Load Dataset

In [9]:
x_train, y_train, x_test, y_test = load_dataset()

170500096/170498071 [==============================] - 2s 0us/step


#### Membangun Model _Fully Connected Neural Network_ dengan spesifikasi minimum memenuhi deskripsi Lab

In [10]:
## Melakukan inisialisasi node per layer
nodes_per_layer = [1000, 1000, 10]
activation_function_per_layer = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers = []
layers.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer)):
  node = nodes_per_layer[i]
  activation = activation_function_per_layer[i]
  layers.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model = Sequential(layers)

## Membangun model
model.build(input_shape=x_train.shape)
model.output_shape

(50000, 10)

#### Melakukan kompilasi ke dalam model

In [11]:
model.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

#### Melakukan Fitting

In [12]:
beginning_hist = model.fit(x_train, y_train, epochs=10, batch_size=5000, validation_data=(x_test,y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 16s 2s/step - loss: 1054907.8750 - accuracy: 0.0991 - recall: 0.0991 - specificity: 0.8999 - f1_score: 0.0991 - val_loss: 2450754.0000 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 2/10
10/10 [==============================] - 15s 2s/step - loss: 400731.0938 - accuracy: 0.0983 - recall: 0.0524 - specificity: 0.9468 - f1_score: 0.0534 - val_loss: 2.3237 - val_accuracy: 0.1000 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 15s 2s/step - loss: 2.3440 - accuracy: 0.1000 - recall: 4.0000e-05 - specificity: 1.0000 - f1_score: 7.9960e-05 - val_loss: 2.3188 - val_accuracy: 0.1000 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 15s 2s/step - loss: 2.3051 - accuracy: 0.1000 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.

#### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [13]:
compare_train_and_test(model, x_train, y_train, x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.1,0.100000
Sensitivity,0.0,0.000000
Specificity,1.0,0.999989
F1 Score,0.0,0.000000


2. Anda dapat memperbaiki hasil yang diperoleh dengan melakukan  (tiga) skenario eksperimen, yaitu: _feature engineering_, menambahkan _layer_, dan _hyperparameter tuning_ (pada classifier).

#### _Feature Engineering_

- Untuk _feature engineering_ pada sesi lab kali ini, akan dilakukan normalisasi input agar berada pada rentang $[0...1]$.

In [14]:
def normalize(x):
  return x / 255.0

normalize_x_train = normalize(x_train)
normalize_x_test = normalize(x_test)

##### Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen _Feature Engineering_

In [15]:
## Melakukan inisialisasi node per layer
nodes_per_layer_for_feature_engineering = [1000, 1000, 10]
activation_function_per_layer_for_feature_engineering = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers_for_feature_engineering = []
layers_for_feature_engineering.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer_for_feature_engineering)):
  node = nodes_per_layer_for_feature_engineering[i]
  activation = activation_function_per_layer_for_feature_engineering[i]
  layers_for_feature_engineering.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model_for_feature_engineering = Sequential(layers_for_feature_engineering)

## Membangun model
model_for_feature_engineering.build(input_shape=x_train.shape)
model_for_feature_engineering.output_shape

(50000, 10)

##### Melakukan kompilasi ke dalam model

In [16]:
model_for_feature_engineering.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

##### Melakukan _fitting_

In [17]:
feature_engineering_hist = model_for_feature_engineering.fit(normalize_x_train, y_train, epochs=10, batch_size=5000, validation_data=(normalize_x_test, y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 16s 2s/step - loss: 2.3496 - accuracy: 0.0949 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.2955 - val_accuracy: 0.1186 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 15s 2s/step - loss: 2.2699 - accuracy: 0.1527 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.2300 - val_accuracy: 0.2030 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 15s 2s/step - loss: 2.2033 - accuracy: 0.2125 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.1749 - val_accuracy: 0.2308 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 15s 2s/step - loss: 2.1538 - accuracy: 0.2409 - recall: 0.0000e+00 - specificity: 1.0000 - f1_sc

#### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [18]:
compare_train_and_test(model_for_feature_engineering, normalize_x_train, y_train, normalize_x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.310140,0.314200
Sensitivity,0.004019,0.004393
Specificity,0.999865,0.999933
F1 Score,0.007749,0.008514


#### Menambahkan _Layer_

- Untuk eksperimen menambahkan _layer_ pada sesi laboratorium kali ini, saya akan menambahkan layer dua buah _hidden layer_ lagi dengan jumlah _node_ sama dengan _hidden layer_ yang lain.

##### Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen Menambahkan _Layer_

In [19]:
## Melakukan inisialisasi node per layer
nodes_per_new_layer = [1000, 1000, 1000, 1000, 10]
activation_function_per_new_layer = [tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
new_layers = []
new_layers.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_new_layer)):
  node = nodes_per_new_layer[i]
  activation = activation_function_per_new_layer[i]
  new_layers.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
new_model = Sequential(new_layers)

## Membangun model
new_model.build(input_shape=x_train.shape)
new_model.output_shape

(50000, 10)

##### Melakukan kompilasi ke dalam model

In [20]:
new_model.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

##### Melakukan _fitting_

In [21]:
new_model_hist = new_model.fit(x_train, y_train, epochs=10, batch_size=5000, validation_data=(x_test, y_test), use_multiprocessing=True)

Epoch 1/10
10/10 [==============================] - 25s 2s/step - loss: nan - accuracy: 0.0987 - recall: nan - specificity: nan - f1_score: nan - val_loss: nan - val_accuracy: 0.1000 - val_recall: nan - val_specificity: nan - val_f1_score: nan
Epoch 2/10
10/10 [==============================] - 25s 2s/step - loss: nan - accuracy: 0.1000 - recall: nan - specificity: nan - f1_score: nan - val_loss: nan - val_accuracy: 0.1000 - val_recall: nan - val_specificity: nan - val_f1_score: nan
Epoch 3/10
10/10 [==============================] - 25s 2s/step - loss: nan - accuracy: 0.1000 - recall: nan - specificity: nan - f1_score: nan - val_loss: nan - val_accuracy: 0.1000 - val_recall: nan - val_specificity: nan - val_f1_score: nan
Epoch 4/10
10/10 [==============================] - 25s 2s/step - loss: nan - accuracy: 0.1000 - recall: nan - specificity: nan - f1_score: nan - val_loss: nan - val_accuracy: 0.1000 - val_recall: nan - val_specificity: nan - val_f1_score: nan
Epoch 5/10
10/10 [======

##### Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [22]:
compare_train_and_test(new_model, x_train, y_train, x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.1,0.1
Sensitivity,NaN,NaN
Specificity,NaN,NaN
F1 Score,NaN,NaN


#### _Hyperparameter Tuning_

- Untuk eksperimen _hyperparameter tuning_ pada sesi laboratorium kali ini, saya akan membagi eksperimen menjadi:
    1. Meningkatkan epoch dari $10$ menjadi $30$.
    2. Menaikkan $\text{learning rate}$ dari $0.001$ menjadi $0.005$.
    3. Menurunkan $\text{learning rate}$ dari $0.001$ menjadi $0.0001$.
    4. Meningkatkan epoch dari $10$ menjadi $30$ dan menaikkan $\text{learning rate}$ dari $0.001$ menjadi $0.005$.
    5. Meningkatkan epoch dari $10$ menjadi $30$ dan menurunkan $\text{learning rate}$ dari $0.001$ menjadi $0.0001$.

- Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen _Hyperparameter Tuning_

In [23]:
## Melakukan inisialisasi node per layer
nodes_per_layer_for_hyperparameter = [1000, 1000, 10]
activation_function_per_layer_for_hyperparameter = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers_for_hyperparameter = []
layers_for_hyperparameter.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer_for_hyperparameter)):
  node = nodes_per_layer_for_hyperparameter[i]
  activation = activation_function_per_layer_for_hyperparameter[i]
  layers_for_hyperparameter.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model_for_hyperparameter = [
  Sequential(layers_for_hyperparameter)
  for _ in range(5) # 5 Mini Scenario
]

## Membangun model
for i in range(5): # 5 Mini Scenario
  model_for_hyperparameter[i].build(input_shape=x_train.shape)

- Melakukan kompilasi ke dalam model

In [24]:
parameter_learning_setiap_skenario = [0.001, 0.005, 0.0001, 0.005, 0.0001]
for i in range(5):
  learning_rate = parameter_learning_setiap_skenario[i]
  model_for_hyperparameter[i].compile(
    optimizer=SGD(lr=learning_rate, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ]      
  )

model_for_hyperparameter_hist = np.zeros(5, dtype=object)

- Melakukan _fitting_

In [25]:
epochs = [30, 10, 10, 30, 30]
for i in range(5):
  model_for_hyperparameter_hist[i] = model_for_hyperparameter[i].fit(x_train, y_train, epochs=epochs[i], batch_size=5000, validation_data=(x_test, y_test), use_multiprocessing=True)

Epoch 1/30
10/10 [==============================] - 15s 2s/step - loss: 122907.3516 - accuracy: 0.0993 - recall: 0.0993 - specificity: 0.8999 - f1_score: 0.0993 - val_loss: 740560.5625 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 2/30
10/10 [==============================] - 15s 1s/step - loss: 92653611489689600.0000 - accuracy: 0.0996 - recall: 0.0996 - specificity: 0.9000 - f1_score: 0.0996 - val_loss: 81669894123852660736.0000 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 3/30
10/10 [==============================] - 15s 1s/step - loss: 8314836782730444800.0000 - accuracy: 0.0994 - recall: 0.0994 - specificity: 0.8999 - f1_score: 0.0994 - val_loss: 9464814.0000 - val_accuracy: 0.1000 - val_recall: 0.1000 - val_specificity: 0.9000 - val_f1_score: 0.1000
Epoch 4/30
10/10 [==============================] - 15s 1s/step - loss: 955383.8750 - accuracy: 0.0996 - recall: 0.0996 - speci

- Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [26]:
val = []
for i in range(5):
  val.append(compare_train_and_test(model_for_hyperparameter[i], x_train, y_train, x_test, y_test, str(i+1)))
pd.concat(val).transpose()

,Data Train 1,Data Test 1,Data Train 2,Data Test 2,Data Train 3,Data Test 3,Data Train 4,Data Test 4,Data Train 5,Data Test 5
Accuracy,0.100020,0.1,0.100020,0.1,0.100020,0.1,0.100020,0.1,0.100020,0.1
Sensitivity,0.000020,0.0,0.000020,0.0,0.000020,0.0,0.000020,0.0,0.000020,0.0
Specificity,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.0
F1 Score,0.000039,0.0,0.000039,0.0,0.000039,0.0,0.000039,0.0,0.000039,0.0


#### Gabungan _Feature Engineering_, Menambahkan _Layer_, dan _Hyperparameter tuning_ ($\text{learning rate} = 0.001$, $\text{epoch} = 30$)

- Membangun Model _Fully Connected Neural Network_ sebagai _dedicated model_ untuk eksperimen _multiple parameter_.

In [27]:
## Melakukan inisialisasi node per layer
nodes_per_layer_for_combined_trick = [1000, 1000, 1000, 1000, 10]
activation_function_per_layer_for_combined_trick = [tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers_for_combined_trick = []
layers_for_combined_trick.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer_for_combined_trick)):
  node = nodes_per_layer_for_combined_trick[i]
  activation = activation_function_per_layer_for_combined_trick[i]
  layers_for_combined_trick.append(keras.layers.Dense(node, activation=activation))

## Mendefinisikan model
model_for_combined_trick = Sequential(layers_for_combined_trick)

## Membangun model
model_for_combined_trick.build(input_shape=x_train.shape)
model_for_combined_trick.output_shape

(50000, 10)

- Melakukan kompilasi ke dalam model

In [28]:
model_for_combined_trick.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

- Melakukan _fitting_

In [29]:
model_for_combined_trick_hist = model_for_combined_trick.fit(normalize_x_train, y_train, epochs=30, batch_size=5000, validation_data=(normalize_x_test, y_test), use_multiprocessing=True)

Epoch 1/30
10/10 [==============================] - 25s 2s/step - loss: 2.3070 - accuracy: 0.0839 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.2928 - val_accuracy: 0.1202 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/30
10/10 [==============================] - 24s 2s/step - loss: 2.2815 - accuracy: 0.1197 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.2675 - val_accuracy: 0.1310 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/30
10/10 [==============================] - 24s 2s/step - loss: 2.2568 - accuracy: 0.1463 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.2429 - val_accuracy: 0.1773 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 4/30
10/10 [==============================] - 24s 2s/step - loss: 2.2329 - accuracy: 0.1958 - recall: 0.0000e+00 - specificity: 1.0000 - f1_sc

- Melakukan Prediksi ke _Data Test_ dan Menampilkan _Metric_-nya

In [30]:
compare_train_and_test(model_for_combined_trick, normalize_x_train, y_train, normalize_x_test, y_test).transpose()

,Data Train,Data Test
Accuracy,0.349980,0.349700
Sensitivity,0.031230,0.032448
Specificity,0.998460,0.998412
F1 Score,0.058135,0.060360


3. Analisis hasil yang anda dapatkan untuk masing-masing skenario.

- Untuk skenario _feature engineering_, normalisasi nilai citra dapat meningkatkan akurasi. Ide normalisasi didasari dengan inputan awalnya bisa pada rentang $[0...255]$, hal ini dapat mengakibatkan beberapa _node_ dapat memiliki nilai yang cenderung sangat kecil dan cenderung sangat besar agar keluaran _node_-nya bisa diantara $[0...1]$. Untuk mengatasi hal itu, saya normalisasi nilainya agar berada pada rentang $[0...1]$.
- Untuk skenario menambahkan _layer_, menambahkan layer sebanyak dua dapat meningkatkan akurasi. Ide menambahkan layer sebanyak dua sebenarnya muncul karena untuk mempercepat proses eksperimen saja dan pengalaman pribadi. Semakin banyak layer, semakin baik karena dapat memodelkan ke tahap yang lebih kompleks. _Tradeoff_-nya semakin banyak layer, semakin lama eksperimen berjalan. Setelah ditambahkan sebanyak
- Untuk skenario melakukan _hyperparameter tuning_, saya hanya melakukan improvisasi pada $\text{Learning Rate}$ dan $\text{Epoch}$-nya saja. Setelah dilakukan mini skenario, _adjustment_ $\text{Learning Rate}$ dan $\text{epoch}$ saja tidak memberikan perubahan yang signifikan dan cenderung sama untuk setiap mini skenario.
- Untuk skenario gabungan ketiganya dan mengambil _hyperparameter tuning_ untuk mini skenario $1$, terlihat bahwa hasil eksperimennya mengalami peningkatan dibanding skenario-skenario yang berdiri sendiri sebelumnya.

### B. _Convolutional Neural Networks_

1. Lakukan Klasifikasi untuk dataset cifar menggunakan _Convolutional Neural Networks_ dengan arsitektur dasar sebagai berikut:
    - _Convolutional Layer_ ($25 \text{ maps}$, $\text{kernel }3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Convolutional Layer ($50 \text{ maps}$, $\text{kernel }3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Hidden Layer ($100 \text{ neuron}$)
    - Activation Function ($\text{ReLU function}$)
    - Output Layer ($10 \text{ class}$)
    - Activation Function ($\text{Softmax function}$)
    - Classification Result
2. Hitung metrik ($\text{accuracy}$, $\text{sensitivity}$ , $\text{specificity}$, dan $f_{1}\text{ score}$) untuk untuk data train
dan data test.

In [31]:
convolutional_layers = []
convolutional_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_layers.append(MaxPooling2D(2,2))
convolutional_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_layers.append(MaxPooling2D(2,2))
convolutional_layers.append(Flatten())
convolutional_layers.append(Dense(100, activation=tf.nn.relu))
convolutional_layers.append(Dense(10, activation=tf.nn.softmax))

convolutional_model = Sequential(convolutional_layers)

## Membangun model
convolutional_model.build(input_shape=x_train.shape)
convolutional_model.output_shape

(50000, 10)

In [32]:
convolutional_model.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

In [ ]:
convolutional_model_hist = convolutional_model.fit(x_train, y_train, epochs=25, batch_size=50, validation_data=(x_test, y_test), use_multiprocessing=True)

Epoch 1/25
 397/1000 [==========>...................] - ETA: 39s - loss: 26.4562 - accuracy: 0.0967 - recall: 0.0012 - specificity: 0.9988 - f1_score: 0.0014

In [ ]:
compare_train_and_test(convolutional_model, x_train, y_train, x_test, y_test).transpose()

3. Anda dapat memperbaiki hasil yang didapat dengan melakukan 2 (dua) skenario eksperimen, yaitu menambah _layer_ dan _hyperparameter tuning_ (pada classifier).

##### Menambahkan _layer_

In [ ]:
convolutional_extra_layers = []
convolutional_extra_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(Reshape(50000, 32, 32, 3))
convolutional_extra_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(Reshape(50000, 32, 32, 3))
convolutional_extra_layers.append(MaxPooling2D(2,2))
convolutional_extra_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(Reshape(50000, 32, 32, 3))
convolutional_extra_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(Reshape(50000, 32, 32, 3))
convolutional_extra_layers.append(MaxPooling2D(2,2))
convolutional_extra_layers.append(Flatten())
convolutional_extra_layers.append(Dense(200, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.HeUniform()))
convolutional_extra_layers.append(Dense(10, activation=tf.nn.softmax))

convolutional_extra_model = Sequential(convolutional_extra_layers)

## Membangun model
convolutional_extra_model.build(input_shape=x_train.shape)
convolutional_extra_model.output_shape

In [ ]:
convolutional_extra_model.compile(
    optimizer=SGD(lr=0.001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

In [155]:
convolutional_extra_model_hist = convolutional_extra_model.fit(x_train, y_train, epochs=50, batch_size=50, validation_data=(x_test, y_test), use_multiprocessing=True)

Epoch 1/50
  67/1000 [=>............................] - ETA: 59s - loss: 369.7099 - accuracy: 0.1030 - recall: 0.0310 - specificity: 0.9714 - f1_score: 0.0315

KeyboardInterrupt: ignored

In [ ]:
compare_train_and_test(convolutional_extra_model, x_train, y_train, x_test, y_test).transpose()